In [1]:
from selenium import webdriver
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd

import requests
from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def getData(url):
    service = Service('C:/Users/alexa/Downloads/chromedriver-win64/chromedriver.exe')
    driver = webdriver.Chrome(service=service)

    driver.get(url)

    main_element = driver.find_element(By.ID, 'mainContent')

    scripts = main_element.find_elements(By.TAG_NAME, 'script')

    json_data = None

    for script in scripts:
        if 'application/ld+json' in script.get_attribute('type'):
            json_data = script.get_attribute('innerHTML')
            break

    driver.quit()

    if json_data:
        data = json.loads(json_data)
    
    else:
        print("pas de json")
        
    return data

In [134]:
def getInfo(data):
    menu = []
    
    for menus in data['hasMenu']:
        for section in menus['hasMenuSection']:
            for item in section['hasMenuItem']:
                menu.append([item['name'],item['description']])
                
    rating = data['aggregateRating']['ratingValue'] 
    priceRange = data['priceRange']
    localisation = data['address']['addressLocality']
    nom = data['name'] 
    coords = [data['geo']['latitude'],data['geo']['longitude']]
    typeCuisine = data['servesCuisine'].split(", ")
    
    resto = pd.DataFrame({"nom" : nom, "typeCuisine" : [typeCuisine], "localisation":localisation, "priceRange":priceRange, "rating":rating, "menu":[menu], "coords":[coords]})
    return resto

In [57]:
url ='https://www.opentable.com/r/market-by-jean-georges-paris?corrid=216daf70-c02b-4cad-8d0c-2af83e2b87ce&avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&p=2&sd=2024-01-09T19%3A00%3A00'
data = getData(url)

{'@id': 'https://www.opentable.com/r/market-by-jean-georges-paris', '@context': 'https://schema.org', '@type': 'Restaurant', 'mainEntityOfPage': 'https://www.jean-georges.com/restaurants/france/paris/market', 'url': 'https://www.jean-georges.com/restaurants/france/paris/market', 'name': 'Market by Jean-Georges', 'description': 'Welcome to Market Restaurant, we look forward to serving you. We are a fashionable restaurant offering, Asian-French cuisine located on avenue Matignon, near the Champs-Elysées. Market was decorated by Christian Liaigre, with simple materials, including marble, wood, and leather. Chef Jean-Georges delivers inventive dishes unsing fresh market produce and original combinations of spices and seasonings.', 'image': ['https://resizer.otstatic.com/v2/photos/xlarge/1/25731878.webp', 'https://resizer.otstatic.com/v2/photos/xlarge/1/25731877.webp', 'https://resizer.otstatic.com/v2/photos/xlarge/1/25731876.webp', 'https://resizer.otstatic.com/v2/photos/xlarge/1/52244348.

In [144]:
restaurants = pd.DataFrame(columns = ["nom", "typeCuisine", "localisation", "priceRange", "rating", "menu", "coords"])

In [145]:
resto = pd.DataFrame({"nom" : nom, "typeCuisine" : [typeCuisine], "localisation":localisation, "priceRange":priceRange, "rating":rating, "menu":[menu], "coords":[coords]})

In [146]:
restaurants = pd.concat([restaurants, resto])

In [147]:
restaurants

,nom,typeCuisine,localisation,priceRange,rating,menu,coords
0,Market by Jean-Georges,"[French, Asian]",Paris,€31 to €50,4.5,"[[Organic Egg Toast, Caviar and Dill , Wine Pa...","[48.870321, 2.312219]"
